**3. Codes for analyzing the Proposed Model**

**3.2 For enhanced generalization capacity**

In [ ]:
import numpy as np

## Number of transmitter-receiver pairs
K = 8

## Minimum rate for the achievable SINR of multiple concurrent transmissions
SINR_P_min = np.array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

## Maximum transmit power
p_max = 1.0

In [ ]:
## Loading a NumPy array from a CSV file
# Loading F_H array from a CSV file
from numpy import loadtxt

## Reading an array from the file
# If we want to read a file from our local drive, we have to first upload it to Collab's session storage.
F_H_2D_L_0dB = np.loadtxt('/content/drive/MyDrive/F_H_2D_0dB.csv', delimiter = ',', dtype = str)
F_H_2D_L_10dB = np.loadtxt('/content/drive/MyDrive/F_H_2D_10dB.csv', delimiter = ',', dtype = str)
F_H_2D_L_20dB = np.loadtxt('/content/drive/MyDrive/F_H_2D_20dB.csv', delimiter = ',', dtype = str)
F_H_2D_L_30dB = np.loadtxt('/content/drive/MyDrive/F_H_2D_30dB.csv', delimiter = ',', dtype = str)
F_H_2D_L_40dB = np.loadtxt('/content/drive/MyDrive/F_H_2D_40dB.csv', delimiter = ',', dtype = str)

# ## Reshaping the array from 2D to 3D
F_H_3D_0dB = F_H_2D_L_0dB.reshape(F_H_2D_L_0dB.shape[0], F_H_2D_L_0dB.shape[1] // K, K)
F_H_3D_10dB = F_H_2D_L_10dB.reshape(F_H_2D_L_10dB.shape[0], F_H_2D_L_10dB.shape[1] // K, K)
F_H_3D_20dB = F_H_2D_L_20dB.reshape(F_H_2D_L_20dB.shape[0], F_H_2D_L_20dB.shape[1] // K, K)
F_H_3D_30dB = F_H_2D_L_30dB.reshape(F_H_2D_L_30dB.shape[0], F_H_2D_L_30dB.shape[1] // K, K)
F_H_3D_40dB = F_H_2D_L_40dB.reshape(F_H_2D_L_40dB.shape[0], F_H_2D_L_40dB.shape[1] // K, K)

F_H_3D_0dB_size = F_H_3D_0dB.shape[0]
F_H_3D_10dB_size = F_H_3D_10dB.shape[0]
F_H_3D_20dB_size = F_H_3D_20dB.shape[0]
F_H_3D_30dB_size = F_H_3D_30dB.shape[0]
F_H_3D_40dB_size = F_H_3D_40dB.shape[0]

In [ ]:
## Function to convert string data to complex data and to remove the initial whitespace
def cnvrt_2_cmplx_data(F_H_3D_size, F_H_3D):
  F_H_list = []
  for k in range(F_H_3D_size):
    for i in range(K):  # Total rows
      for j in range(K):  # Total columns
        F_H_temp = complex(F_H_3D[k][i][j].strip())
        F_H_list.append(F_H_temp)
  F_H_array = np.array(F_H_list)
  F_H = F_H_array.reshape((F_H_3D_size, K, K)) # H_size X row X column_count
  return F_H

In [ ]:
## Converting string data to complex data and removing the initial whitespace
F_H_0dB = cnvrt_2_cmplx_data(F_H_3D_0dB_size, F_H_3D_0dB)
F_H_10dB = cnvrt_2_cmplx_data(F_H_3D_10dB_size, F_H_3D_10dB)
F_H_20dB = cnvrt_2_cmplx_data(F_H_3D_20dB_size, F_H_3D_20dB)
F_H_30dB = cnvrt_2_cmplx_data(F_H_3D_30dB_size, F_H_3D_30dB)
F_H_40dB = cnvrt_2_cmplx_data(F_H_3D_40dB_size, F_H_3D_40dB)

print(F_H_0dB.shape)
print(F_H_10dB.shape)
print(F_H_20dB.shape)
print(F_H_30dB.shape)
print(F_H_40dB.shape)

F_H_0dB_size = F_H_0dB.shape[0]
F_H_10dB_size = F_H_10dB.shape[0]
F_H_20dB_size = F_H_20dB.shape[0]
F_H_30dB_size = F_H_30dB.shape[0]
F_H_40dB_size = F_H_40dB.shape[0]

# print(F_H_0dB)
# print(F_H_10dB)
# print(F_H_20dB)
# print(F_H_30dB)
# print(F_H_40dB)

(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)


In [ ]:
import numba as nb

## Function to compute the square of the absolute value of an array of complex numbers
@nb.vectorize([nb.float64(nb.complex128),nb.float32(nb.complex64)])
def cmplx_abs_sqr(cmplx_var):
  return cmplx_var.real**2 + cmplx_var.imag**2

In [ ]:
## Function to generate the matrix A (K x K)
def generate_A(F_H_size, K, SINR_P_min, F_H):
  Aij_list = []
  F_H_abs_sqr = cmplx_abs_sqr(F_H)

  for k in range(F_H_size):
    for i in range(K):  # Total rows
      Aj_list =[]
      for j in range(K): # Total columns
        if i==j:
          A = F_H_abs_sqr[k,i,j]
        else:
          A = np.multiply(-SINR_P_min[i], F_H_abs_sqr[k,i,j])
        Aj_list.append(A)
      Aij_list.append(Aj_list)
  Aij_array = np.array(Aij_list)
  Aij = Aij_array.reshape((F_H_size, K, K)) # H_size X row X column
  return Aij

In [ ]:
## Create matrix A
A_0dB = generate_A(F_H_0dB_size, K, SINR_P_min, F_H_0dB)
A_10dB = generate_A(F_H_10dB_size, K, SINR_P_min, F_H_10dB)
A_20dB = generate_A(F_H_20dB_size, K, SINR_P_min, F_H_20dB)
A_30dB = generate_A(F_H_30dB_size, K, SINR_P_min, F_H_30dB)
A_40dB = generate_A(F_H_40dB_size, K, SINR_P_min, F_H_40dB)

print(A_0dB.shape)
print(A_10dB.shape)
print(A_20dB.shape)
print(A_30dB.shape)
print(A_40dB.shape)

# print(A_0dB)
# print(A_10dB)
# print(A_20dB)
# print(A_30dB)
# print(A_40dB)

(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)


In [ ]:
## Variances for noise signals
sigma_sqr_noise_0dB = np.array([1e-0, 1e-0, 1e-0, 1e-0, 1e-0, 1e-0, 1e-0, 1e-0], dtype = float)
sigma_sqr_noise_10dB = np.array([1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1], dtype = float)
sigma_sqr_noise_20dB = np.array([1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2], dtype = float)
sigma_sqr_noise_30dB = np.array([1e-3, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3], dtype = float)
sigma_sqr_noise_40dB = np.array([1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4], dtype = float)

In [ ]:
## Function to generate the vector b (K x 1)
def generate_b(F_H_size, K, SINR_P_min, sigma_sqr_noise, F_H):
  bi_list = []
  for k in range(F_H_size):
    for i in range(K):  # Total rows, i.e., total transmitters
      b = np.multiply(SINR_P_min[i], sigma_sqr_noise[i])
      bi_list.append(b)
  bi_array = np.array(bi_list)
  bi = bi_array.reshape((F_H_size, K, 1)) # H_size X row X column
  return bi

In [ ]:
## Create vector b
b_0dB = generate_b(F_H_0dB_size, K, SINR_P_min, sigma_sqr_noise_0dB, F_H_0dB)
b_10dB = generate_b(F_H_10dB_size, K, SINR_P_min, sigma_sqr_noise_10dB, F_H_10dB)
b_20dB = generate_b(F_H_20dB_size, K, SINR_P_min, sigma_sqr_noise_20dB, F_H_20dB)
b_30dB = generate_b(F_H_30dB_size, K, SINR_P_min, sigma_sqr_noise_30dB, F_H_30dB)
b_40dB = generate_b(F_H_40dB_size, K, SINR_P_min, sigma_sqr_noise_40dB, F_H_40dB)

print(b_0dB.shape)
print(b_10dB.shape)
print(b_20dB.shape)
print(b_30dB.shape)
print(b_40dB.shape)

# print(b_0dB)
# print(b_10dB)
# print(b_20dB)
# print(b_30dB)
# print(b_40dB)

(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)


In [ ]:
## Create matrix A_inv, i.e., the pseudo inverse of matrix A
A_inv_0dB = np.linalg.pinv(A_0dB)
A_inv_10dB = np.linalg.pinv(A_10dB)
A_inv_20dB = np.linalg.pinv(A_20dB)
A_inv_30dB = np.linalg.pinv(A_30dB)
A_inv_40dB = np.linalg.pinv(A_40dB)

A_inv_0dB[A_inv_0dB<0] = 0
A_inv_10dB[A_inv_10dB<0] = 0
A_inv_20dB[A_inv_20dB<0] = 0
A_inv_30dB[A_inv_30dB<0] = 0
A_inv_40dB[A_inv_40dB<0] = 0

print(A_inv_0dB.shape)
print(A_inv_10dB.shape)
print(A_inv_20dB.shape)
print(A_inv_30dB.shape)
print(A_inv_40dB.shape)

# print(A_inv_0dB)
# print(A_inv_10dB)
# print(A_inv_20dB)
# print(A_inv_30dB)
# print(A_inv_40dB)

(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)
(250000, 8, 8)


In [ ]:
## Create a vector p_hat = (A_inv x b)
p_hat_0dB = np.matmul(A_inv_0dB, b_0dB)
p_hat_10dB = np.matmul(A_inv_10dB, b_10dB)
p_hat_20dB = np.matmul(A_inv_20dB, b_20dB)
p_hat_30dB = np.matmul(A_inv_30dB, b_30dB)
p_hat_40dB = np.matmul(A_inv_40dB, b_40dB)

print(p_hat_0dB.shape)
print(p_hat_10dB.shape)
print(p_hat_20dB.shape)
print(p_hat_30dB.shape)
print(p_hat_40dB.shape)

# print(p_hat_0dB)
# print(p_hat_10dB)
# print(p_hat_20dB)
# print(p_hat_30dB)
# print(p_hat_40dB)

(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)


In [ ]:
## Convert p_max_array to (K x 1) vector
p_max_array = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], dtype = float)
p_max_vector = p_max_array.reshape((K, 1)) # row X column
print(p_max_vector)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [ ]:
## Create a vector X = (p_max_vector - p_hat)
X_0dB = p_max_vector - p_hat_0dB
X_10dB = p_max_vector - p_hat_10dB
X_20dB = p_max_vector - p_hat_20dB
X_30dB = p_max_vector - p_hat_30dB
X_40dB = p_max_vector - p_hat_40dB

print(X_0dB.shape)
print(X_10dB.shape)
print(X_20dB.shape)
print(X_30dB.shape)
print(X_40dB.shape)

# print(X_0dB)
# print(X_10dB)
# print(X_20dB)
# print(X_30dB)
# print(X_40dB)

(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)


In [ ]:
## Function to generate a vector beta = MIN[(p_max_vector - p_hat) / A_inv_cv]
def generate_beta(F_H_size, A_inv, X):
  beta_list = []

  for k in range(F_H_size):
    for i in range(K):  # Total columns
      ak = A_inv[k,:,i]
      akr = ak.reshape((K, 1)) # row X column
      with np.errstate(divide='ignore'):
        beta_w = np.where(akr != 0.0, np.divide(X[k], akr), np.inf)
        # beta_w = np.divide(X[k], akr)
      # beta_w = np.divide(X[k], akr)
      beta_min = np.amin(beta_w)
      beta_list.append(beta_min)

  beta_array = np.array(beta_list)
  beta = beta_array.reshape((F_H_size, K, 1)) # H_size X row X column_count
  return beta

In [ ]:
## Generate a vector beta = MIN[(p_max_vector - p_hat) / A_inv_cv]
beta_0dB = generate_beta(F_H_0dB_size, A_inv_0dB, X_0dB)
beta_10dB = generate_beta(F_H_10dB_size, A_inv_10dB, X_10dB)
beta_20dB = generate_beta(F_H_20dB_size, A_inv_20dB, X_20dB)
beta_30dB = generate_beta(F_H_30dB_size, A_inv_30dB, X_30dB)
beta_40dB = generate_beta(F_H_40dB_size, A_inv_40dB, X_40dB)

print(beta_0dB.shape)
print(beta_10dB.shape)
print(beta_20dB.shape)
print(beta_30dB.shape)
print(beta_40dB.shape)

beta_0dB_size = beta_0dB.shape[0]
beta_10dB_size = beta_10dB.shape[0]
beta_20dB_size = beta_20dB.shape[0]
beta_30dB_size = beta_30dB.shape[0]
beta_40dB_size = beta_40dB.shape[0]

# print(beta_0dB)
# print(beta_10dB)
# print(beta_20dB)
# print(beta_30dB)
# print(beta_40dB)

(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)
(250000, 8, 1)


In [ ]:
## Function to split datasets for training, validation, and testing.
def split(np_array):
  # data_size = np_array.shape[0]
  # train_data_size = int(data_size * 0.8)
  # valid_data_size = int(data_size * 0.1)
  # test_data_size = int(data_size * 0.1)

  train_data_size = int(200000)
  valid_data_size = int(25000)
  test_data_size = int(25000)

  train_e_indx = train_data_size
  valid_e_indx = train_e_indx + valid_data_size
  test_e_indx = valid_e_indx + test_data_size
  test_data_size_n = test_e_indx - valid_e_indx

  row_count = np_array.shape[1]
  column_count = np_array.shape[2]

  train_data = np.empty((train_data_size, row_count, column_count), dtype = complex, order = 'C')
  valid_data = np.empty((valid_data_size, row_count, column_count), dtype = complex, order = 'C')
  test_data = np.empty((test_data_size_n, row_count, column_count), dtype = complex, order = 'C')

  for i in range(train_e_indx):
    train_data[i] = np_array[i]

  xv = 0
  for j in range(train_e_indx, valid_e_indx):
    valid_data[xv] = np_array[j]
    xv = xv + 1

  xt = 0
  for k in range(valid_e_indx, test_e_indx):
    test_data[xt] = np_array[k]
    xt = xt + 1

  # print(train_data.shape, valid_data.shape, test_data.shape)


  ## Training input will be the absolute value
  train_input = np.absolute(train_data)
  valid_input = np.absolute(valid_data)
  test_input = np.absolute(test_data)

  print(train_input.shape, valid_input.shape, test_input.shape)

  return [train_input, valid_input, test_input, test_data]

In [ ]:
## Split F_H matrix
F_H_S_0dB = split(F_H_0dB)
train_input_F_H_0dB = F_H_S_0dB[0]
valid_input_F_H_0dB = F_H_S_0dB[1]
test_input_F_H_0dB = F_H_S_0dB[2]
test_data_F_H_0dB = F_H_S_0dB[3]

F_H_S_10dB = split(F_H_10dB)
train_input_F_H_10dB = F_H_S_10dB[0]
valid_input_F_H_10dB = F_H_S_10dB[1]
test_input_F_H_10dB = F_H_S_10dB[2]
test_data_F_H_10dB = F_H_S_10dB[3]

F_H_S_20dB = split(F_H_20dB)
train_input_F_H_20dB = F_H_S_20dB[0]
valid_input_F_H_20dB = F_H_S_20dB[1]
test_input_F_H_20dB = F_H_S_20dB[2]
test_data_F_H_20dB = F_H_S_20dB[3]

F_H_S_30dB = split(F_H_30dB)
train_input_F_H_30dB = F_H_S_30dB[0]
valid_input_F_H_30dB = F_H_S_30dB[1]
test_input_F_H_30dB = F_H_S_30dB[2]
test_data_F_H_30dB = F_H_S_30dB[3]

F_H_S_40dB = split(F_H_40dB)
train_input_F_H_40dB = F_H_S_40dB[0]
valid_input_F_H_40dB = F_H_S_40dB[1]
test_input_F_H_40dB = F_H_S_40dB[2]
test_data_F_H_40dB = F_H_S_40dB[3]

(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)


In [ ]:
## Split A_inv matrix
A_inv_S_0dB = split(A_inv_0dB)
train_input_A_inv_0dB = A_inv_S_0dB[0]
valid_input_A_inv_0dB = A_inv_S_0dB[1]
test_input_A_inv_0dB = A_inv_S_0dB[2]
test_data_A_inv_0dB = A_inv_S_0dB[3]

A_inv_S_10dB = split(A_inv_10dB)
train_input_A_inv_10dB = A_inv_S_10dB[0]
valid_input_A_inv_10dB = A_inv_S_10dB[1]
test_input_A_inv_10dB = A_inv_S_10dB[2]
test_data_A_inv_10dB = A_inv_S_10dB[3]

A_inv_S_20dB = split(A_inv_20dB)
train_input_A_inv_20dB = A_inv_S_20dB[0]
valid_input_A_inv_20dB = A_inv_S_20dB[1]
test_input_A_inv_20dB = A_inv_S_20dB[2]
test_data_A_inv_20dB = A_inv_S_20dB[3]

A_inv_S_30dB = split(A_inv_30dB)
train_input_A_inv_30dB = A_inv_S_30dB[0]
valid_input_A_inv_30dB = A_inv_S_30dB[1]
test_input_A_inv_30dB = A_inv_S_30dB[2]
test_data_A_inv_30dB = A_inv_S_30dB[3]

A_inv_S_40dB = split(A_inv_40dB)
train_input_A_inv_40dB = A_inv_S_40dB[0]
valid_input_A_inv_40dB = A_inv_S_40dB[1]
test_input_A_inv_40dB = A_inv_S_40dB[2]
test_data_A_inv_40dB = A_inv_S_40dB[3]

(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)
(200000, 8, 8) (25000, 8, 8) (25000, 8, 8)


In [ ]:
## Split b vector
b_S_0dB = split(b_0dB)
train_input_b_0dB = b_S_0dB[0]
valid_input_b_0dB = b_S_0dB[1]
test_input_b_0dB = b_S_0dB[2]
test_data_b_0dB = b_S_0dB[3]

b_S_10dB = split(b_10dB)
train_input_b_10dB = b_S_10dB[0]
valid_input_b_10dB = b_S_10dB[1]
test_input_b_10dB = b_S_10dB[2]
test_data_b_10dB = b_S_10dB[3]

b_S_20dB = split(b_20dB)
train_input_b_20dB = b_S_20dB[0]
valid_input_b_20dB = b_S_20dB[1]
test_input_b_20dB = b_S_20dB[2]
test_data_b_20dB = b_S_20dB[3]

b_S_30dB = split(b_30dB)
train_input_b_30dB = b_S_30dB[0]
valid_input_b_30dB = b_S_30dB[1]
test_input_b_30dB = b_S_30dB[2]
test_data_b_30dB = b_S_30dB[3]

b_S_40dB = split(b_40dB)
train_input_b_40dB = b_S_40dB[0]
valid_input_b_40dB = b_S_40dB[1]
test_input_b_40dB = b_S_40dB[2]
test_data_b_40dB = b_S_40dB[3]

(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)


In [ ]:
## Split X vector
X_S_0dB = split(X_0dB)
train_input_X_0dB = X_S_0dB[0]
valid_input_X_0dB = X_S_0dB[1]
test_input_X_0dB = X_S_0dB[2]
test_data_X_0dB = X_S_0dB[3]

X_S_10dB = split(X_10dB)
train_input_X_10dB = X_S_10dB[0]
valid_input_X_10dB = X_S_10dB[1]
test_input_X_10dB = X_S_10dB[2]
test_data_X_10dB = X_S_10dB[3]

X_S_20dB = split(X_20dB)
train_input_X_20dB = X_S_20dB[0]
valid_input_X_20dB = X_S_20dB[1]
test_input_X_20dB = X_S_20dB[2]
test_data_X_20dB = X_S_20dB[3]

X_S_30dB = split(X_30dB)
train_input_X_30dB = X_S_30dB[0]
valid_input_X_30dB = X_S_30dB[1]
test_input_X_30dB = X_S_30dB[2]
test_data_X_30dB = X_S_30dB[3]

X_S_40dB = split(X_40dB)
train_input_X_40dB = X_S_40dB[0]
valid_input_X_40dB = X_S_40dB[1]
test_input_X_40dB = X_S_40dB[2]
test_data_X_40dB = X_S_40dB[3]

(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)


In [ ]:
## Split beta vector
beta_S_0dB = split(beta_0dB)
train_input_beta_0dB = beta_S_0dB[0]
valid_input_beta_0dB = beta_S_0dB[1]
test_input_beta_0dB = beta_S_0dB[2]
test_data_beta_0dB = beta_S_0dB[3]

beta_S_10dB = split(beta_10dB)
train_input_beta_10dB = beta_S_10dB[0]
valid_input_beta_10dB = beta_S_10dB[1]
test_input_beta_10dB = beta_S_10dB[2]
test_data_beta_10dB = beta_S_10dB[3]

beta_S_20dB = split(beta_20dB)
train_input_beta_20dB = beta_S_20dB[0]
valid_input_beta_20dB = beta_S_20dB[1]
test_input_beta_20dB = beta_S_20dB[2]
test_data_beta_20dB = beta_S_20dB[3]

beta_S_30dB = split(beta_30dB)
train_input_beta_30dB = beta_S_30dB[0]
valid_input_beta_30dB = beta_S_30dB[1]
test_input_beta_30dB = beta_S_30dB[2]
test_data_beta_30dB = beta_S_30dB[3]

beta_S_40dB = split(beta_40dB)
train_input_beta_40dB = beta_S_40dB[0]
valid_input_beta_40dB = beta_S_40dB[1]
test_input_beta_40dB = beta_S_40dB[2]
test_data_beta_40dB = beta_S_40dB[3]

(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)


In [ ]:
## Split p_hat vector
p_hat_S_0dB = split(p_hat_0dB)
train_input_p_hat_0dB = p_hat_S_0dB[0]
valid_input_p_hat_0dB = p_hat_S_0dB[1]
test_input_p_hat_0dB = p_hat_S_0dB[2]
test_data_p_hat_0dB = p_hat_S_0dB[3]

p_hat_S_10dB = split(p_hat_10dB)
train_input_p_hat_10dB = p_hat_S_10dB[0]
valid_input_p_hat_10dB = p_hat_S_10dB[1]
test_input_p_hat_10dB = p_hat_S_10dB[2]
test_data_p_hat_10dB = p_hat_S_10dB[3]

p_hat_S_20dB = split(p_hat_20dB)
train_input_p_hat_20dB = p_hat_S_20dB[0]
valid_input_p_hat_20dB = p_hat_S_20dB[1]
test_input_p_hat_20dB = p_hat_S_20dB[2]
test_data_p_hat_20dB = p_hat_S_20dB[3]

p_hat_S_30dB = split(p_hat_30dB)
train_input_p_hat_30dB = p_hat_S_30dB[0]
valid_input_p_hat_30dB = p_hat_S_30dB[1]
test_input_p_hat_30dB = p_hat_S_30dB[2]
test_data_p_hat_30dB = p_hat_S_30dB[3]

p_hat_S_40dB = split(p_hat_40dB)
train_input_p_hat_40dB = p_hat_S_40dB[0]
valid_input_p_hat_40dB = p_hat_S_40dB[1]
test_input_p_hat_40dB = p_hat_S_40dB[2]
test_data_p_hat_40dB = p_hat_S_40dB[3]

(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)
(200000, 8, 1) (25000, 8, 1) (25000, 8, 1)


In [ ]:
## Create EsN0 vector
EsN0_array_0dB = np.full(shape = F_H_0dB_size, fill_value = 0, dtype = int)
EsN0_array_10dB = np.full(shape = F_H_10dB_size, fill_value = 10, dtype = int)
EsN0_array_20dB = np.full(shape = F_H_20dB_size, fill_value = 20, dtype = int)
EsN0_array_30dB = np.full(shape = F_H_30dB_size, fill_value = 30, dtype = int)
EsN0_array_40dB = np.full(shape = F_H_40dB_size, fill_value = 40, dtype = int)

EsN0_vector_0dB = EsN0_array_0dB.reshape((F_H_0dB_size, 1)) # row X column
EsN0_vector_10dB = EsN0_array_10dB.reshape((F_H_10dB_size, 1)) # row X column
EsN0_vector_20dB = EsN0_array_20dB.reshape((F_H_20dB_size, 1)) # row X column
EsN0_vector_30dB = EsN0_array_30dB.reshape((F_H_30dB_size, 1)) # row X column
EsN0_vector_40dB = EsN0_array_40dB.reshape((F_H_40dB_size, 1)) # row X column

print(EsN0_vector_0dB.shape)
print(EsN0_vector_10dB.shape)
print(EsN0_vector_20dB.shape)
print(EsN0_vector_30dB.shape)
print(EsN0_vector_40dB.shape)

(250000, 1)
(250000, 1)
(250000, 1)
(250000, 1)
(250000, 1)


In [ ]:
## Function to split EsN0 vector for training, validation, and testing.
def split_EsN0(np_vector):
  # data_size = np_vector.shape[0]
  # train_data_size = int(data_size * 0.8)
  # valid_data_size = int(data_size * 0.1)
  # test_data_size = int(data_size * 0.1)

  train_data_size = int(200000)
  valid_data_size = int(25000)
  test_data_size = int(25000)

  train_e_indx = train_data_size
  valid_e_indx = train_e_indx + valid_data_size
  test_e_indx = valid_e_indx + test_data_size
  test_data_size_n = test_e_indx - valid_e_indx

  row_count = np_vector.shape[1]
  column_count = 1

  train_data = np.empty((train_data_size, row_count, column_count), dtype = int, order = 'C')
  valid_data = np.empty((valid_data_size, row_count, column_count), dtype = int, order = 'C')
  test_data = np.empty((test_data_size_n, row_count, column_count), dtype = int, order = 'C')

  for i in range(train_e_indx):
    train_data[i] = np_vector[i]

  xv = 0
  for j in range(train_e_indx, valid_e_indx):
    valid_data[xv] = np_vector[j]
    xv = xv + 1

  xt = 0
  for k in range(valid_e_indx, test_e_indx):
    test_data[xt] = np_vector[k]
    xt = xt + 1

  # print(train_data.shape, valid_data.shape, test_data.shape)


  ## Training input will be the absolute value
  train_input = np.absolute(train_data)
  valid_input = np.absolute(valid_data)
  test_input = np.absolute(test_data)

  print(train_input.shape, valid_input.shape, test_input.shape)

  return [train_input, valid_input, test_input, test_data]

In [ ]:
## Split EsN0 vector
EsN0_S_0dB = split_EsN0(EsN0_vector_0dB)
train_input_EsN0_0dB = EsN0_S_0dB[0]
valid_input_EsN0_0dB = EsN0_S_0dB[1]
test_input_EsN0_0dB = EsN0_S_0dB[2]
test_data_EsN0_0dB = EsN0_S_0dB[3]

EsN0_S_10dB = split_EsN0(EsN0_vector_10dB)
train_input_EsN0_10dB = EsN0_S_10dB[0]
valid_input_EsN0_10dB = EsN0_S_10dB[1]
test_input_EsN0_10dB = EsN0_S_10dB[2]
test_data_EsN0_10dB = EsN0_S_10dB[3]

EsN0_S_20dB = split_EsN0(EsN0_vector_20dB)
train_input_EsN0_20dB = EsN0_S_20dB[0]
valid_input_EsN0_20dB = EsN0_S_20dB[1]
test_input_EsN0_20dB = EsN0_S_20dB[2]
test_data_EsN0_20dB = EsN0_S_20dB[3]

EsN0_S_30dB = split_EsN0(EsN0_vector_30dB)
train_input_EsN0_30dB = EsN0_S_30dB[0]
valid_input_EsN0_30dB = EsN0_S_30dB[1]
test_input_EsN0_30dB = EsN0_S_30dB[2]
test_data_EsN0_30dB = EsN0_S_30dB[3]

EsN0_S_40dB = split_EsN0(EsN0_vector_40dB)
train_input_EsN0_40dB = EsN0_S_40dB[0]
valid_input_EsN0_40dB = EsN0_S_40dB[1]
test_input_EsN0_40dB = EsN0_S_40dB[2]
test_data_EsN0_40dB = EsN0_S_40dB[3]

(200000, 1, 1) (25000, 1, 1) (25000, 1, 1)
(200000, 1, 1) (25000, 1, 1) (25000, 1, 1)
(200000, 1, 1) (25000, 1, 1) (25000, 1, 1)
(200000, 1, 1) (25000, 1, 1) (25000, 1, 1)
(200000, 1, 1) (25000, 1, 1) (25000, 1, 1)


In [ ]:
## Creating datasets for training
train_input_F_H = np.concatenate((train_input_F_H_0dB, train_input_F_H_10dB,
                                  train_input_F_H_20dB, train_input_F_H_30dB,
                                  train_input_F_H_40dB,), axis=0)

train_input_EsN0 = np.concatenate((train_input_EsN0_0dB, train_input_EsN0_10dB,
                                   train_input_EsN0_20dB, train_input_EsN0_30dB,
                                   train_input_EsN0_40dB), axis=0)

train_input_A_inv = np.concatenate((train_input_A_inv_0dB, train_input_A_inv_10dB,
                                    train_input_A_inv_20dB, train_input_A_inv_30dB,
                                    train_input_A_inv_40dB), axis=0)

train_input_X = np.concatenate((train_input_X_0dB, train_input_X_10dB,
                                train_input_X_20dB, train_input_X_30dB,
                                train_input_X_40dB), axis=0)

train_input_beta = np.concatenate((train_input_beta_0dB, train_input_beta_10dB,
                                   train_input_beta_20dB, train_input_beta_30dB,
                                   train_input_beta_40dB), axis=0)

train_input_p_hat = np.concatenate((train_input_p_hat_0dB, train_input_p_hat_10dB,
                                    train_input_p_hat_20dB, train_input_p_hat_30dB,
                                    train_input_p_hat_40dB), axis=0)

print(train_input_F_H.shape)
print(train_input_EsN0.shape)
print(train_input_A_inv.shape)
print(train_input_X.shape)
print(train_input_p_hat.shape)

(1000000, 8, 8)
(1000000, 1, 1)
(1000000, 8, 8)
(1000000, 8, 1)
(1000000, 8, 1)


In [ ]:
## Creating datasets for validation
valid_input_F_H = np.concatenate((valid_input_F_H_0dB, valid_input_F_H_10dB,
                                  valid_input_F_H_20dB, valid_input_F_H_30dB,
                                  valid_input_F_H_40dB,), axis=0)

valid_input_EsN0 = np.concatenate((valid_input_EsN0_0dB, valid_input_EsN0_10dB,
                                   valid_input_EsN0_20dB, valid_input_EsN0_30dB,
                                   valid_input_EsN0_40dB), axis=0)

valid_input_A_inv = np.concatenate((valid_input_A_inv_0dB, valid_input_A_inv_10dB,
                                    valid_input_A_inv_20dB, valid_input_A_inv_30dB,
                                    valid_input_A_inv_40dB), axis=0)

valid_input_X = np.concatenate((valid_input_X_0dB, valid_input_X_10dB,
                                valid_input_X_20dB, valid_input_X_30dB,
                                valid_input_X_40dB), axis=0)

valid_input_beta = np.concatenate((valid_input_beta_0dB, valid_input_beta_10dB,
                                   valid_input_beta_20dB, valid_input_beta_30dB,
                                   valid_input_beta_40dB), axis=0)

valid_input_p_hat = np.concatenate((valid_input_p_hat_0dB, valid_input_p_hat_10dB,
                                    valid_input_p_hat_20dB, valid_input_p_hat_30dB,
                                    valid_input_p_hat_40dB), axis=0)

print(valid_input_F_H.shape)
print(valid_input_EsN0.shape)
print(valid_input_A_inv.shape)
print(valid_input_X.shape)
print(valid_input_p_hat.shape)

(125000, 8, 8)
(125000, 1, 1)
(125000, 8, 8)
(125000, 8, 1)
(125000, 8, 1)


In [ ]:
## Shuffling the training datasets
train_shuffler = np.random.permutation(len(train_input_F_H))
train_input_F_H_shuffled = train_input_F_H[train_shuffler]
train_input_EsN0_shuffled = train_input_EsN0[train_shuffler]
train_input_A_inv_shuffled = train_input_A_inv[train_shuffler]
train_input_X_shuffled = train_input_X[train_shuffler]
train_input_beta_shuffled = train_input_beta[train_shuffler]
train_input_p_hat_shuffled = train_input_p_hat[train_shuffler]

In [ ]:
## Shuffling the validation datasets
valid_shuffler = np.random.permutation(len(valid_input_F_H))
valid_input_F_H_shuffled = valid_input_F_H[valid_shuffler]
valid_input_EsN0_shuffled = valid_input_EsN0[valid_shuffler]
valid_input_A_inv_shuffled = valid_input_A_inv[valid_shuffler]
valid_input_X_shuffled = valid_input_X[valid_shuffler]
valid_input_beta_shuffled = valid_input_beta[valid_shuffler]
valid_input_p_hat_shuffled = valid_input_p_hat[valid_shuffler]

In [ ]:
## Reshaping train_input_F_H_shuffled and adding train_input_EsN0_shuffled
const = K*K
len1 = train_input_F_H_shuffled.shape[0]
train_input_F_H_shuffled_reshaped = train_input_F_H_shuffled.reshape((len1, 1, const)) # size X row X column
train_y_true = np.concatenate((train_input_F_H_shuffled_reshaped, train_input_EsN0_shuffled), axis=2)
print(train_y_true.shape)

(1000000, 1, 65)


In [ ]:
## Reshaping train_input_F_H_shuffled and adding train_input_EsN0_shuffled
len2 = valid_input_F_H_shuffled.shape[0]
valid_input_F_H_shuffled_reshaped = valid_input_F_H_shuffled.reshape((len2, 1, const)) # size X row X column
valid_y_true = np.concatenate((valid_input_F_H_shuffled_reshaped, valid_input_EsN0_shuffled), axis=2)
print(valid_y_true.shape)

(125000, 1, 65)


In [ ]:
## Define the DNN model - The Functional API
import tensorflow as tf
from tensorflow import keras
## from tensorflow.keras import layers # shows warning
from keras.api._v2.keras import layers
from keras.layers import Input, concatenate, Lambda
from keras.models import Model


hij_inputs = keras.Input(shape=(K,K), name = "hij_inputs")
f1 = layers.Flatten(name = "flatten_layer_hij")(hij_inputs)

EsN0_inputs = keras.Input(shape=(1,1), name = "EsN0_inputs")
f2 = layers.Flatten(name = "flatten_layer_EsN0")(EsN0_inputs)

concat_layers = concatenate([f1, f2])

d1 = layers.Dense(2*K*K, activation="relu", name = "dense_layer_1")(concat_layers)
b1 = layers.BatchNormalization(name = "batch_norm_layer_1")(d1)

d2 = layers.Dense(K*K, activation="relu", name = "dense_layer_2")(b1)
b2 = layers.BatchNormalization(name = "batch_norm_layer_2")(d2)

# meu = layers.Dense(K, activation="relu", name = "meu")(b2)
meu = layers.Dense(K, activation="sigmoid", name = "meu")(b2)

A_inv_inputs = keras.Input(shape=(K,K), name = "A_inv_inputs")
f3 = layers.Flatten(name = "flatten_layer_A_inv")(A_inv_inputs)

X_inputs = keras.Input(shape=(K,1), name = "X_inputs")
f4 = layers.Flatten(name = "flatten_layer_X")(X_inputs)

beta_inputs = keras.Input(shape=(K,1), name = "beta_inputs")
f5 = layers.Flatten(name = "flatten_layer_beta")(beta_inputs)

p_hat_inputs = keras.Input(shape=(K,1), name = "p_hat_inputs")
f6 = layers.Flatten(name = "flatten_layer_p_hat")(p_hat_inputs)

def custom_layer(tensor):
  t_A_inv = tensor[0]
  t_X = tensor[1]
  t_beta = tensor[2]
  t_p_hat = tensor[3]
  t_meu = tensor[4]

  A_inv_cl = tf.reshape(t_A_inv[:,0:K*K], (-1,K,K))
  X_cl = tf.reshape(t_X[:,0:K*1], (-1,K,1))
  beta_cl = tf.reshape(t_beta[:,0:K*1], (-1,K,1))
  p_hat_cl = tf.reshape(t_p_hat[:,0:K*1], (-1,K,1))
  meu_cl = tf.reshape(t_meu[:,0:K*1], (-1,K,1))

  meu_ewm = tf.math.multiply(beta_cl, meu_cl)

  alpha_dnumr = tf.matmul(A_inv_cl, meu_ewm)
  alpha_whole = tf.divide(X_cl, alpha_dnumr)
  alpha = tf.reduce_min(alpha_whole, axis = 1, keepdims = True)
  max_p = tf.constant([1.0])
  alpha = tf.math.minimum(max_p, alpha)
  meu_P = tf.multiply(meu_ewm, alpha)

  Z_cl = tf.matmul(A_inv_cl, meu_P)
  P_hat_cl = tf.add(p_hat_cl, Z_cl)
  P_hat_cl_Norm = tf.math.divide(P_hat_cl, tf.reduce_max(P_hat_cl, axis = 1, keepdims = True))

  # return P_hat_cl
  return P_hat_cl_Norm

lambda_layer = tf.keras.layers.Lambda(custom_layer, name="lambda_layer")([f3, f4, f5, f6, meu])
f7 = layers.Flatten(name = "flatten_layer_output")(lambda_layer)

model = keras.Model(inputs = [hij_inputs, EsN0_inputs, A_inv_inputs, X_inputs, beta_inputs, p_hat_inputs], outputs = f7, name = "functional_api")
model.summary()

Model: "functional_api"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 hij_inputs (InputLayer)        [(None, 8, 8)]       0           []                               
                                                                                                  
 EsN0_inputs (InputLayer)       [(None, 1, 1)]       0           []                               
                                                                                                  
 flatten_layer_hij (Flatten)    (None, 64)           0           ['hij_inputs[0][0]']             
                                                                                                  
 flatten_layer_EsN0 (Flatten)   (None, 1)            0           ['EsN0_inputs[0][0]']            
                                                                                     

In [ ]:
## Plot the model as a graph
# keras.utils.plot_model(model, "Functional_API_Model.png")

In [ ]:
## Display the input and output shapes of each layer
# keras.utils.plot_model(model, "Functional_API_Model_with_shape_info.png", show_shapes=True)

In [ ]:
## The customized loss function

def custom_loss(y_true, y_pred):
  # p = y_pred
  p = tf.math.multiply(p_max, y_pred)

  mtrx_elmnt = K*K
  EsN0_val = y_true[0][0][mtrx_elmnt]
  y_true_updt = y_true[:,:,:-1]

  if EsN0_val < 10:
    sigma_sqr_noise_lf = 1e-0
  elif EsN0_val >= 10 and EsN0_val < 20:
    sigma_sqr_noise_lf = 1e-1
  elif EsN0_val >= 20 and EsN0_val < 30:
    sigma_sqr_noise_lf = 1e-2
  elif EsN0_val >= 30 and EsN0_val < 40:
    sigma_sqr_noise_lf = 1e-3
  else:
    sigma_sqr_noise_lf = 1e-4

  hij = tf.reshape(y_true_updt[:,0:K*K], (-1,K,K))
  hij_abs_sqr = tf.math.square(tf.math.abs(hij))

  R_P = 0.0
  for i in range(K):  # Total rows
    ph = 0.0
    for j in range(K):  # Total columns
      ph_j = tf.math.multiply(p[:,j], hij_abs_sqr[:,i,j])
      ph = tf.math.add(ph, ph_j)

    numr = tf.math.multiply(p[:,i], hij_abs_sqr[:,i,i])
    dnumr = tf.math.add(sigma_sqr_noise_lf, tf.math.subtract(ph, numr))
    SINR_i = tf.math.divide(numr, dnumr)
    R_P = tf.math.add(R_P, (tf.math.log(1 + SINR_i)/tf.math.log(2.0)))

  loss = -R_P
  loss = tf.reduce_mean(loss) # batch mean
  return loss

In [ ]:
## Build and compile the DNN model
## Training and Testing
import matplotlib.pyplot as plt

optA = tf.keras.optimizers.Adam(learning_rate = 0.0001)
# optA = tf.keras.optimizers.Adam(learning_rate = 0.0001, clipnorm=0.92)
model.compile(optimizer = optA, loss = custom_loss)

train_input = [train_input_F_H_shuffled, train_input_EsN0_shuffled, train_input_A_inv_shuffled,
               train_input_X_shuffled, train_input_beta_shuffled, train_input_p_hat_shuffled]

valid_input = [valid_input_F_H_shuffled, valid_input_EsN0_shuffled, valid_input_A_inv_shuffled,
               valid_input_X_shuffled, valid_input_beta_shuffled, valid_input_p_hat_shuffled]

history = model.fit(train_input, train_y_true, epochs = 50,
                    validation_data = (valid_input, valid_y_true), batch_size = 1000)

# plt.plot(history.epoch, history.history['loss'], color = "blue", label = "Training")
# plt.plot(history.epoch, history.history['val_loss'], color="black", label = "Validation")
# plt.xlabel("epochs")
# plt.ylabel("loss")
# plt.legend()
# plt.show()

Epoch 1/50
1000/1000 [==============================] - 14s 10ms/step - loss: -2.5855 - val_loss: -2.6343
Epoch 2/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.6801 - val_loss: -2.6506
Epoch 3/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.6860 - val_loss: -2.6549
Epoch 4/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.6842 - val_loss: -2.6576
Epoch 5/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.7222 - val_loss: -2.6975
Epoch 6/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.7248 - val_loss: -2.7028
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.7476 - val_loss: -2.7048
Epoch 8/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.7337 - val_loss: -2.7057
Epoch 9/50
1000/1000 [==============================] - 8s 8ms/step - loss: -2.7528 - val_loss: -2.7064
Epoch 10/50
1000/1000 [==============================] - 8s 8m

In [ ]:
## Constraint violation probability and
## finding indexes of test_input_F_H matrix with the hij set that do not satisfy
## constraint on the minimum SINR_P_min rate but satisfy the maximum transmit
## power p_max

test_input = [test_input_F_H_0dB, test_input_EsN0_0dB, test_input_A_inv_0dB,
              test_input_X_0dB, test_input_beta_0dB, test_input_p_hat_0dB]
# output_P_hat_temp = model.predict(test_input)
output_P_hat_temp = np.multiply(p_max, model.predict(test_input))
output_P_hat = output_P_hat_temp.reshape((output_P_hat_temp.shape[0], output_P_hat_temp.shape[1], 1)) # test_input_F_H_size X row X column
output_P_hat_size = output_P_hat.shape[0]
test_data_F_H_abs_sqr = cmplx_abs_sqr(test_data_F_H_0dB)

indx_n = []
count_v = 0

for k in range(output_P_hat_size):
  for i in range(K):  # Total rows
    ph = 0
    for j in range(K):  # Total columns
      ph_j = np.multiply(output_P_hat[k,j], test_data_F_H_abs_sqr[k,i,j])
      ph = ph + ph_j

    numr = np.multiply(output_P_hat[k,i], test_data_F_H_abs_sqr[k,i,i])
    dnumr = sigma_sqr_noise_0dB[i] + ph - numr
    SINR_out = np.divide(numr, dnumr)

    if np.round(SINR_out, decimals= 3) < SINR_P_min[i]:
      indx_n.append(k)
      count_v = count_v + 1
      # print(SINR_out)
      break

violation_prb = (count_v / output_P_hat_size) * 100
print("Constraints Violation Probability: {:.2f}%".format(violation_prb))
# print(len(indx_n))
# print(indx_n)

782/782 [==============================] - 1s 1ms/step
Constraints Violation Probability: 0.00%


In [ ]:
## Function to calculate the average sum rate
# Here, p_model is the output of DNN, and it is a 2D array.
import math

def average_sum_rate(hij, p_model, sigma_sqr_noise, K):
  R = 0
  hij_size = hij.shape[0]
  hij_abs_sqr = cmplx_abs_sqr(hij)

  for k in range(hij_size):
    for i in range(K):  # Total rows
      phn = 0
      for j in range(K):  # Total columns
        phn_j = np.multiply(p_model[k,j], hij_abs_sqr[k,i,j])
        phn = phn + phn_j

      numr_s = np.multiply(p_model[k,i], hij_abs_sqr[k,i,i])
      dnumr_s = sigma_sqr_noise[i] + phn - numr_s
      R_temp = math.log2(1 + np.divide(numr_s, dnumr_s))
      R = R + R_temp

  return (R/hij_size)

In [ ]:
## DNN Sum Rate for test_data_F_H
sumrate_F_H = average_sum_rate(test_data_F_H_0dB, output_P_hat, sigma_sqr_noise_0dB, K)
print("Average Sum Rate for all H matrices: {:.3f} Bit/Second/Hertz".format(sumrate_F_H))

Average Sum Rate for all H matrices: 2.605 Bit/Second/Hertz


In [ ]:
## Checking (A_inv x b), i.e., the power for negative values
count_n = 0
for c in range(output_P_hat_size):
  p_temp = np.matmul(test_input_A_inv_0dB[c], test_input_b_0dB[c])
  if np.any(p_temp < 0):
    count_n = count_n + 1
    print(c,'\n')
    print(p_temp)

print(count_n)

0


In [ ]:
## Checking P_hat, i.e., the power for test_data_F_H for negative values
## and Hit Rate i.e. percentage for 0 <= P_hat <= p_max
count_p = 0
count_n = 0

for n in range(output_P_hat_size):
  P_max = np.amax(output_P_hat[n])
  if np.round(P_max, decimals = 3) <= 1:
    count_p = count_p + 1

  if np.any(output_P_hat[n] < 0):
    count_n = count_n + 1
    print(n,'\n')
    print(output_P_hat)

p_hit_rate = (count_p / output_P_hat_size) * 100
print("Htt Rate for Power : {:.2f}%".format(p_hit_rate))
print("Negative power count: ", count_n)

Htt Rate for Power : 100.00%
Negative power count:  0
